### OPEN API 를 이용한 챗봇 시스템 

In [1]:
!pip show openapi
# !pip install openai

Name: openapi
Version: 2.0.0
Summary: Python OpenAPI 2.0 (Swagger) object model
Home-page: https://github.com/globality-corp/openapi
Author: Globality Engineering
Author-email: engineering@globality.com
License: 
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: inflection, jsonschema
Required-by: 


---
### OPENAI API 호출 예제 

In [2]:
from openai import OpenAI
from pprint import pprint
import os
api_key=os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

client = OpenAI(api_key=api_key)
model = 'gpt-3.5-turbo-1106'

messages = [
    {'role':'system','content': 'You are a helphful assistant.'},
    {'role':'user','content': '군자역 혼잡도 알려줘?'}
]
response = client.chat.completions.create(model=model, messages=messages).model_dump()
pprint(response)

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'content': '죄송합니다. 현재는 군자역의 혼잡도 정보를 제공할 수 없습니다. 혼잡도 '
                                     '정보는 지역 및 교통 상황에 따라 변할 수 있습니다. 혼잡도를 확인하려면 '
                                     '현지 교통 정보나 관련 애플리케이션을 활용하시거나, 현지 교통 관리 '
                                     '기관에 문의하시는 것이 도움이 될 수 있습니다.',
                          'function_call': None,
                          'role': 'assistant',
                          'tool_calls': None}}],
 'created': 1721799390,
 'id': 'chatcmpl-9oOxqPTq6csd1IZuRA6gv2wChFWul',
 'model': 'gpt-3.5-turbo-1106',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_b81a85d4e3',
 'usage': {'completion_tokens': 136, 'prompt_tokens': 36, 'total_tokens': 172}}


> gpt 는 json type 으로 답을 넘겨준다. 

---
### 

 * [model dump ]
    : 메서드 체이닝 -> 메서드를 연쇄적으로 호출하는 방식 

---
### ai chatbot 을 만들기 위해서는 프롬프트 엔지니어링이 필요하다.

In [3]:
prompt = """
내일 다음 주제로 미래 교육 포럼에서 발표할 발제문을 작성해야합니다.
발제문의 주요 내용을 불릿 기호로 간략히 정리해 주세요.
주제:{agenda}


""".format(agenda="인공지능 시대의 교육의 역활과 의미")
model = 'gpt-3.5-turbo-1106'
response = client.chat.completions.create(
    model= model,
    messages=[{'role':'user','content':prompt}]
).to_dict
pprint(response)



<bound method BaseModel.to_dict of ChatCompletion(id='chatcmpl-9oOxsy9TTAgZF1BWhSmRAunWEoOCB', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='- 인공지능 기술의 발전과 교육\n- 학교교육과 인공지능의 융합\n- 학생들의 미래 직업과 교육의 관련성\n- 교사와 교육자의 역할과 변화\n- 윤리적인 측면에서의 교육과 인공지능\n- 인공지능을 효과적으로 활용한 교육 방법론의 개발\n- 인간 중심의 교육과 인공지능의 조화적 결합 방안', role='assistant', function_call=None, tool_calls=None))], created=1721799392, model='gpt-3.5-turbo-1106', object='chat.completion', service_tier=None, system_fingerprint='fp_b81a85d4e3', usage=CompletionUsage(completion_tokens=166, prompt_tokens=97, total_tokens=263))>


In [4]:
from pprint import pprint

prompt = """
내일 다음 주제로 미래 교육 포럼에서 발표할 발제문을 작성해야합니다.
발제문의 주요 내용을 불릿 기호로 간략히 정리해 주세요.
주제:{agenda}
""".format(agenda="인공지능 시대의 교육의 역할과 의미")

model = 'gpt-3.5-turbo-1106'
response = client.chat.completions.create(
    model=model,
    messages=[{'role': 'user', 'content': prompt}]
)

pprint(response.choices[0].message.content)


('- 인공지능 기술의 발전으로 교육 시스템이 변화하는 현상\n'
 '- 교육의 목표와 방향성에 대한 새로운 고민\n'
 '- 인공지능이 적절히 활용될 수 있는 교육 방법과 방향 탐구\n'
 '- 미래 교육의 핵심가치와 인간적 요소 강화를 통한 새로운 교육 패러다임 제시')


---
### Chater 5 프롬프트 엔지니어링의 핵심 기법 
- few shot
- Cot
-SC
- ToT
- ReAct
-RAG


In [5]:
def GPT_prompt(model = 2, template_go = False , prompt=""):
    from pprint import pprint
    import json
    from openai import OpenAI
    from pprint import pprint
    import os
    api_key=os.environ.get("OPENAI_API_KEY")
    client = OpenAI(api_key=api_key)
    
    if template_go :
        
        template = """
        당신은 번역함수이며, 반환값은 반드시 JSON 데이터 여야 합니다. 
        STEP 별로 작업을 수행하면서 그 결과를 아래의 출력 결과 JSON 포맷에 작성하세요.
        STEP-1. 아래 세 개의 백틱으로 구분된 텍스트를 원문 그대로 읽어 올것
        STEP-2. 입력받은 텍스트가 긍정적이라면 True를 표기할것
        STEP-3. 다음의 말투로 번역할 것: ["지구의 나이는 45억살이야.","세종 대왕은 조선의 위대한 국왕이야."]
        ```{text}```
        ---
        출력결과 : {{"STEP-1":<입력텍스트>, "STEP-2": <true/false>, "STEP-3":<번역결과>}}
        """
        text = "William Shakespeare was an English playwrit, poet and actor.  He is widely regarded as the greatest writer in the English language and the world's pre-eminent dramatist."
        
        template = template.format(text=text)
        context = [{"role": "user","content": template}]
        model_1 = 'gpt-3.5-turbo-1106'
        model_2 = 'gpt-4-0613'
        response = client.chat.completions.create(
            model=model_2 if model == 2 else model_1,
            messages=context,
            temperature=0,
            top_p=0,
            seed=1234
        ).model_dump()
        pprint(json.loads(response['choices'][0]['message']['content']))
        return json.loads(response['choices'][0]['message']['content'])
    else :
        template = prompt
        context = [{"role": "user","content": template}]

        model_1 = 'gpt-3.5-turbo-1106'
        model_2 = 'gpt-4-0613'
        response = client.chat.completions.create(
            model=model_2 if model == 2 else model_1,
            messages=context,
            temperature=0,
            top_p=0,
            seed=1234
        ).model_dump()
        print(response['choices'][0]['message']['content'])
    
    
    # pprint(response.choices[0].message.content)
GPT_prompt(prompt="너 바보야?")

저는 인공지능으로, 학습과 개선을 통해 더 나은 서비스를 제공하려 노력하고 있습니다. 만약 제가 잘못한 부분이 있다면, 구체적으로 알려주시면 감사하겠습니다.


---
###  프롬프트 가이드라인
- 과업배경: 당신은 번역함수
- 원하는 결과 예시 
- 단계별 가이드 
- 제약사항 
- 수행결과 작성 형식 : 출력결과 : {{"STEP-1":<입력텍스트>, "STEP-2": <true/false>, "STEP-3":<번역결과>}}
- 적절한 구분자의 사용
- 구체적이면서 명확한 과업지시


---
#### 인컨텍스트 러닝이란?
- 학습햇던 내용을 바탕으로 답하는 것이 아니라, 런타임 시점에 제공되는 입력정보를 바탕으로 추로하는 능력 : 주어진 문맥안에서 그 의미를 배운다
- 언어모델 추론능력 향상 -> 프롬프트 엔지니어링의 출발 

- Few-shot Prompting 
    : 그 질문과 관련된 예시를 적절히 제공하면 언어 모델이 그 예시의 의미를 추론하여 답을 말하는원리

## Zero shot prompting 

In [6]:
## Zero shot prompting 
from Functions import NLP_Service as N
template = """
긍정 또는 부정으로 답변을 작성하세요.
Q: 매력적인 이성과 사랑에 빠졌어!
A: 
"""

N.GPT_prompt(prompt=template)

'긍정'


In [7]:
## Few shot prompting 
from Functions import NLP_Service as N
template = """
아래 예시를 참조해 마지막 답변을 긍정 또는 부정으로 작성하세요

```
Q: 난 오늘 기분이 나빠 
A: 긍정
```
Q: 드디서 사업에 성공했어
A: 부정
```
Q: 요즘 너무 행복해
A: 부정
```
Q: 슬픈일이 벌어졌어
A: 긍정
```
Q: 매력적인 이성과 사랑에 빠졌어!
A: < 정답을 작성하고 그렇게 답한 이유를 말하세요>

"""
N.GPT_prompt(prompt=template)


'A: 부정'


---
### JSON 모드 대답 ( 2023.11.6 도입)

In [8]:
def GPT_prompt_json(model = 2, template_go = False , prompt="", json_mode=True):
    from pprint import pprint
    import json
    from openai import OpenAI
    from pprint import pprint
    import os
    api_key=os.environ.get("OPENAI_API_KEY")
    client = OpenAI(api_key=api_key)
    model_1 = 'gpt-3.5-turbo-1106'
    model_2 = 'gpt-4-0613'
    context = [{"role": "user","content": "한국 초대 대통령 이름, 출생일자, 사망일자를 json 으로 기술하세요"}]
    completion = client.chat.completions.create(
        model= model_1,
        messages = context,
        response_format ={"type":"json_object"}
    )
    print(json.loads(completion.choices[0].message.content))
    
    # pprint(response.choices[0].message.content)
GPT_prompt(prompt="너 바보야?")

저는 인공지능으로, 감정이나 지식을 가지고 있지 않습니다. 그러므로 바보라는 개념이 적용되지 않습니다. 도움이 필요하시다면 언제든지 말씀해주세요.


---
### 챗봇 시스템 구현 
- 대화를 주고 받는 멀티턴 방식 
- gpt ->완성형, 대화형 언어모델이 있음. 
- gpt3.5 는 완성형 모델  
#### 챗봇 시스템 설계 


---
### 2024.07.24 : page 172  시작 
 - 책에서 flask 를 사용하는데 aws 가 아니라 구름에서 사용함. => 해볼것 